Make a function with the following parameters: 
1. video input 
2.threshold 
3. similarity measure
4. granularity 
Output: smoothed list 

another function: 
1. takes the smoothed list of distances as input 
2. output is the histogram across videos 


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift
import os 
#import seaborn as sns
from itertools import zip_longest
from tqdm import tqdm
import matplotlib.lines as mlines

In [ ]:
os.chdir('/Users/andreibirladeanu/Documents/Data/videos_trial')

In [ ]:
cap = cv2.VideoCapture('1043_meal.mp4')
frames = []
counter = 0
differences = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #converts captured frame to Grayscale for easier analysis
    blur = cv2.GaussianBlur(gray,(5,5),0)
    frames.append(gray)
    if counter > 20:
#         difference = np.abs(frames[counter] - frames[counter-1])
        
        difference = np.abs(cv2.subtract(frames[counter], frames[counter-21]))
        #if  np.sum(difference)  > 6880109:
         #   cv2.imshow('difference',difference)
          #  cv2.waitKey(0)
        
        ret2,th = cv2.threshold(difference,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        #if  np.sum(th)  < 6880109:
         #   cv2.imshow('th',th)
          #  cv2.waitKey(0)
        differences.append(np.sum(th))
        #cv2.imshow('difference',th)
        #cv2.waitKey(0)

    
#     differences.append(th.sum()
    #differences.append(np.sum(th))

    counter = counter + 1
    
    

cap.release()
cv2.destroyAllWindows()

In [ ]:
def frame_differencing(video_input, threshold_method,granularity):
    cap = cv2.VideoCapture(video_input)
    frames = []
    counter = 0
    differences = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #converts captured frame to Grayscale for easier analysis
        blur = cv2.GaussianBlur(gray,(5,5),0)
        frames.append(gray)
        if counter > granularity:
            difference = np.abs(cv2.subtract(frames[counter], frames[counter-granularity+1]))
            ret2,th = cv2.threshold(difference,0,255,threshold_method)
            differences.append(np.sum(th))
        counter = counter + 1
    differences=np.array(differences)
    differences = differences[np.where(differences < np.percentile(differences, 98))[0]]
    cap.release()
    cv2.destroyAllWindows()
    return differences



In [ ]:
trial = frame_differencing('1043_meal.mp4',cv2.THRESH_BINARY+cv2.THRESH_OTSU, 10)

In [ ]:
len(trial)

In [ ]:
plt.plot(trial)

In [ ]:

path = '/Users/andreibirladeanu/Documents/Data/videos_trial'

videos = [file for file in sorted(os.listdir(path))]


In [ ]:
def dataframes(videos, threshold_method, granularity):
    frame_diffs = []
    for video in tqdm(videos): 
        frame_diffs.append(frame_differencing(video,threshold_method,granularity))
    frame_diffs = np.array(frame_diffs, dtype='object')
    data = pd.DataFrame(zip_longest(*frame_diffs), columns = [video.split('_')[0] for video in videos])
    data_rad = dataframe[dataframe.columns & [str(x) for x in rad]]
    data_no_rad = dataframe[dataframe.columns & [str(x) for x in no_rad]]
    plt.rcParams['font.size']=30
    #figure(facecolor='grey')
    plt.rcParams['axes.facecolor'] = 'white'
    plt.rcParams["figure.figsize"] = [50,25]
    #plt.rc('legend',fontsize=20)
    
    fig, ax = plt.subplots()
    for x in data_rad:  
        n, bins, patches= ax.hist(data_rad[x], bins='fd',  color='red', alpha=.9, stacked=True)
    
    for y in data_no_rad:
        n, bins, patches= plt.hist(data_no_rad[y], bins='fd',  color='green', alpha=.9, stacked=True)
    #plt.legend(loc=2, prop={'size': 6})
    eight = mlines.Line2D([], [], color='red', marker='o', ls='', label='rad', markersize = 9)
    nine = mlines.Line2D([], [], color='green', marker='o', ls='', label='no-rad', markersize = 9)
    # etc etc
    ax.legend(handles=[eight, nine], markerscale=7, fontsize='x-large')
    
    #plt.axvline(0.36, color='k', linestyle='dashed', linewidth=5)
    #plt.text(0.25, 250, ' <5%', size='large')
    #plt.xlim(right = 1)
    plt.grid(True)
    plt.xlabel("Frame change")
    plt.ylabel("Probability (%)")
    plt.title("Video frame changes across all videos")
    plt.show()
    return data

In [ ]:
dataframe = dataframes(videos, cv2.THRESH_BINARY+cv2.THRESH_OTSU, 10)

In [ ]:
dataframe

In [ ]:
dataframe.plot()


In [ ]:
rad = [1047, 1059, 1069, 1079, 1089, 1093, 1099, 1107, 1108, 
      1117, 1122, 1124, 1125, 1129, 1131, 1132, 1134, 1148, 1186, 
      1190, 1195, 1206, 1210, 1217, 1230, 1246, 1250, 1264, 2009, 2027]
no_rad = [1043, 1049, 1053, 1062, 1073, 1080, 1082, 1091, 1092, 
         1096, 1097, 1098, 1104, 1105, 1112, 1118, 1130, 1146, 
         1161, 1170, 1180, 1181, 1184, 1188, 1234, 1241, 1245, 1263,
         1269, 1282, 2025]
data_rad = dataframe[dataframe.columns & [str(x) for x in rad]]
data_rad
data_no_rad = dataframe[dataframe.columns & [str(x) for x in no_rad]]
data_no_rad

In [ ]:
import matplotlib.lines as mlines
plt.rcParams['font.size']=30
#figure(facecolor='grey')
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams["figure.figsize"] = [50,25]
#plt.rc('legend',fontsize=20)

fig, ax = plt.subplots()
for x in data_rad:  
    n, bins, patches= ax.hist(data_rad[x], bins='fd',  color='red', alpha=.9, stacked=True)

for y in data_no_rad:
    n, bins, patches= plt.hist(data_no_rad[y], bins='fd',  color='green', alpha=.9, stacked=True)
#plt.legend(loc=2, prop={'size': 6})
eight = mlines.Line2D([], [], color='red', marker='o', ls='', label='rad', markersize = 9)
nine = mlines.Line2D([], [], color='green', marker='o', ls='', label='no-rad', markersize = 9)
# etc etc
ax.legend(handles=[eight, nine], markerscale=7, fontsize='x-large')

#plt.axvline(0.36, color='k', linestyle='dashed', linewidth=5)
#plt.text(0.25, 250, ' <5%', size='large')
#plt.xlim(right = 1)
plt.grid(True)
plt.xlabel("Frame change")
plt.ylabel("Probability (%)")
plt.title("Video frame changes across all videos")
plt.show()
#fig.savefig('histogram_meal.png')